In [1]:
#     US_Biogas_Analysis.ipynb
#     McKay Rytting
#     This notebook is an initial exploration of US Biogas data compiled by USDA

In [263]:
import pandas as pd
import numpy as np

In [9]:
# Read the data file in and list all of its tabs:
allBiogasData = pd.read_excel('BioGas_Download.xlsx', sheet_name=None)
allBiogasData.keys()

dict_keys(['Readme', 'Agriculture BioGas System', 'Landfill BioGas System', 'Wastewater BioGas System', 'Food Scrap BioGas System', 'Compressed NG Fueling Stations', 'Electric Fueling Stations', 'NG Pipelines Gathering', 'NG Pipelines Interstate', 'NG Pipelines Intrastate', '2013 Electricity Prices', '2014 Gas Prices', 'State Incentives Total Count', 'Net Metering Rules', 'Net Metering Caps', 'Livestock Dairy Inventory', 'Livestock Poultry Inventory', 'Livestock Swine Inventory', 'Crop Residue Bagasse ', 'Crop Residue Barley Straw', 'Crop Residue Cornstover', 'Crop Residue Sorghum', 'Crop Residue Rice', 'Crop Residue Wheatstraw', 'Energy Crops Cane', 'Energy Crops Miscanthus', 'Energy Crops Switchgrass', 'NRCS EQIP Practice Standards'])

In [4]:
# Create individual dataframes for tabs of interest
dfAgriculture=allBiogasData['Agriculture BioGas System']
dfLandfill=allBiogasData['Landfill BioGas System']
dfIncentives=allBiogasData['State Incentives Total Count']
dfDairyInventory=allBiogasData['Livestock Dairy Inventory']
dfSwineInventory=allBiogasData['Livestock Swine Inventory']
dfPoultryInventory=allBiogasData['Livestock Poultry Inventory']

In [5]:
# Clean up livestock inventory dataframes, combine into single dataframe of livestock counts in US counties
dfLivestockInventory=pd.merge(dfDairyInventory,dfSwineInventory, how='outer',
                              left_on=['County','State'], right_on=['NAME','STATE NAME'])
dfLivestockInventory=pd.merge(dfLivestockInventory,dfPoultryInventory, how='outer',
                              left_on=['NAME','STATE NAME'], right_on=['NAME','STATE NAME'])
dfLivestockInventory.drop(columns=['NAME','STATE NAME'],inplace=True)
dfLivestockInventory.rename(columns={'Milk Cow Inventory':'Cow Inventory',
                                     'Inventory Total Higs and Pigs':'Swine Inventory'},
                           inplace=True)
dfLivestockInventory=dfLivestockInventory.fillna(0)

In [25]:
# Want to clean up the CoDigestion column in the agriculture dataframe. First need a list of unique values to turn into categories.
dfAgriculture['CoDigestion']=dfAgriculture['CoDigestion'].fillna('None')
dfAgriculture['CoDigestion'].unique()

array(['None', 'Waste water (from cheese plant)',
       'Wastewater (warm clean up water from cheese plant)',
       'Cheese whey, sudan grass, and residuals (30 tons/day sudan silage, 20 tons/day whey)',
       'Sludge (Paper sludge substrate)', 'Additional substrates',
       'Organic Wastes',
       'Crop wastes, food wastes, haylage, and cooking grease',
       'Wastes from surrounding community', 'Food waste',
       'Crude glycerine (from biodiesel plant)',
       'Food Processing Waste (Syrup stillage from ethanol plant; Crude glycerine from biodiesel plant)',
       'Cheese whey', 'Organic wastes (silage leachate and food waste)',
       'Slaughterhouse wastewater',
       'Food processing waste (cheese whey, waste onions, and potato starch water)',
       'Food processing waste (food waste from grapes; milk/ice cream and salad dressing production)',
       'Food waste/organic waste',
       'Organic food waste and agricultural residue',
       'Food wastes (Milk processing wa

In [245]:
# Make masks for various types of CoDigestion
foodmask=dfAgriculture['CoDigestion'].str.contains('food|Food')
watermask=dfAgriculture['CoDigestion'].str.contains('water|Water')
agmask=dfAgriculture['CoDigestion'].str.contains('agri|Agri')
mixmask=dfAgriculture['CoDigestion'].str.contains('mix|Mix')
allmask=dfAgriculture['CoDigestion']=='None'
multmask=(foodmask^watermask==agmask)

In [257]:
# Apply masks to dataframe
dfAgriculture['Codigestion'][allmask]='None'
dfAgriculture['Codigestion'][~allmask]='Other'
dfAgriculture['Codigestion'][multmask^~foodmask]='Mult'
dfAgriculture['Codigestion'][agmask & ~multmask]='Agricultural Substrates'
dfAgriculture['Codigestion'][watermask & ~multmask]='Wastewater'
dfAgriculture['Codigestion'][foodmask & ~multmask]='Food'
dfAgriculture['Codigestion'][mixmask]='Mix'

In [259]:
# Check to make sure things look categorized correctly!
dfAgriculture[dfAgriculture['Codigestion']=='Mult'][['CoDigestion','Codigestion']]

,CoDigestion,Codigestion
78,"Food processing waste (cheese whey, waste onio...",Mult
87,Organic food waste and agricultural residue,Mult
123,"Food wastes (molasses processor wash water, Je...",Mult
141,Agricultural substrates (fish processing waste...,Mult
151,"Agricultural substrates (waste grain, food was...",Mult


In [265]:
# Rename Codigestion columns to make a bit more sense
dfAgriculture.rename(columns={'CoDigestion':'CoDigestion Details'})
dfAgriculture.head()

,Name,Full Address,City,State,Year Opened,Year Closed,Start Date,Owner,Developer,Emission Reductions MMTCO2E/Year,Digester Typer,Farm Type,Population Feeding Digester,CoDigestion,Biogas Generation Estimate (Cubic Feet a Day),Biogas End Use,Boiler Capacity (BTU an hour),Baseline System,Codigestion
0,Cargill - Sandy River Farm,"Morrilton, AR United States",Morrilton,AR,2008,NaN,NaN,NaN,NaN,2847.284662,Covered Lagoon,Swine,"4,200",None,NaN,Flared Full Time,NaN,Storage Lagoon,None
1,Bob Giacomini Dairy,"Point Reyes Station, CA United States",Point Reyes Station,CA,2009,NaN,NaN,NaN,Williams Engineering Associates,1592.636260,Covered Lagoon,Dairy,300,Waste water (from cheese plant),"25,000",Cogeneration,NaN,Storage Lagoon,Wastewater
2,Bullfrog Dairy,"Imperial, CA United States",Imperial,CA,2008,NaN,NaN,NaN,"RCM International, LLC",17518.998860,Covered Lagoon,Dairy,"3,300",None,NaN,Electricity,NaN,Storage Lagoon,None
3,CAL-Denier Dairy,"Galt, CA United States",Galt,CA,2008,NaN,NaN,NaN,"RCM International, LLC",3983.482409,Covered Lagoon,Dairy,900,None,"33,000",Electricity,NaN,Storage Lagoon,None
4,Castelanelli Bros. Dairy,"Lodi, CA United States",Lodi,CA,2004,NaN,NaN,NaN,"RCM International, LLC; Cover installed by Env...",12582.925030,Covered Lagoon,Dairy,"3,213",None,"89,148",Electricity,NaN,Storage Lagoon,None


In [266]:
# What are the potential end uses?
dfAgriculture['Biogas End Use'].unique()

array(['Flared Full Time', 'Cogeneration', 'Electricity',
       'Cogeneration; Boiler/Furnace Fuel', 'Electricity; CNG', nan,
       'Boiler/Furnace Fuel', 'Electricity; Boiler/Furnace Fuel',
       'Cogeneration; CNG', 'Cogeneration; Electricity',
       'Boiler/Furnace Fuel; Electricity', 'Pipeline Gas', 'electricity'],
      dtype=object)

In [ ]:
# Want only one entry in end use column (Flared, Heat, Electricity, CNG, Pipeline), consider making second end use column?